<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/Clase_Aprendizaje_Profundo/blob/main/L11_01_Autoresgresor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizaje profundo
##Dr. Carlos Villaseñor
### Modelo autoregresivo de series de tiempo


Paso 1. Cargar las siguientes bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Paso 2. Cargue los datos

In [ ]:
data = pd.read_csv('daily-min-temperatures.csv')
print(data.head())

Paso 3. Grafiquemos los datos

In [ ]:
x = np.asanyarray(data[['Temp']])
plt.plot(x)

Paso 4. Investiguemos la correlación entre un dato y su pasdo

In [ ]:
delay = 1
plt.scatter(x[delay:], x[:-delay])
print(np.corrcoef(x[delay:].transpose(), x[:-delay].transpose()))

Paso 5. Generalizando lo anterior para un número creciente de retardos, obtenemos la gráfica de autocorrelación

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(data.Temp)

Paso 6. Las series de tiempo necesitan una partición de datos especifica.

In [ ]:
# Calcular número de datos
n = data.Temp.shape[0]
test_size = 0.1
train_set =  data.Temp[:int((1-test_size) * n)]
test_set =  data.Temp[-int(test_size* n):]

print('Train size: ', train_set.shape)
print('Test size: ', test_set.shape)

plt.plot(np.arange(0,train_set.shape[0]), train_set, '-b')
plt.plot(np.arange(train_set.shape[0],train_set.shape[0]+ test_set.shape[0]),
         test_set, '-r')

Paso 7. Para poder entrenar fuera de linea podemos expandir el dataset, tomando el autovector y los valores deseados. El siguiente código hace esta expansión

In [ ]:
def prepare_data(x, delays, steps):
  data = pd.DataFrame(x)
  for i in range(1, delays + steps):
      data = pd.concat([data, x.shift(-i)], axis=1)
  data = data[:-(delays + steps)]
  x = np.asanyarray(data.iloc[:, 0:delays])
  y = np.asanyarray(data.iloc[:, delays:])
  return x, y

print('Original:\n', train_set)
x, y = prepare_data(train_set, 20, 1)
print('x:\n', x)
print('y:\n', y)

Paso 8. Entrenemos un modelo de regresión lineal

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y)

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(xtrain, ytrain.ravel())

print('Train: ', model.score(xtrain, ytrain.ravel()))
print('Test: ', model.score(xtest, ytest))

Paso 9. Veamos como se comporta para datos nuevos

In [ ]:
x, y = prepare_data(test_set, 20, 1)
ypred = model.predict(x)

plt.plot(y, '-m')
plt.plot(ypred, '-g')
print(y.shape)